In [140]:
import os
import warnings
import importlib
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, confusion_matrix, f1_score, accuracy_score, \
    precision_score, recall_score
import xgboost as xgb
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, confusion_matrix, f1_score
from sklearn.model_selection import KFold, LeavePGroupsOut
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib import pyplot as plt
import matplotlib
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from tabpfn import TabPFNClassifier
from sklearn.preprocessing import label_binarize
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.utils.class_weight import compute_sample_weight
import shap
from sktime.transformations.panel.rocket import Rocket

%matplotlib inline
# matplotlib.use('TkAgg')
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler  # install via: pip install imbalanced-learn

warnings.filterwarnings('ignore')
import torch
from scipy.stats import skew
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

seed = 69
torch.manual_seed(seed)
np.random.seed(seed)

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


segment_length = 3
label = 'agitation-next'
kfold = True
weighted = True
weighted = True

results_dir = '/home/ali/PycharmProjects/tihm/results/results-06h-binary-prediction-03-sequential-kfold-weighted'



True
NVIDIA GeForce RTX 3080 Ti Laptop GPU


In [141]:
root = '/home/ali/PycharmProjects/tihm/dataset'

dataset_06h = pd.read_csv(os.path.join(root, 'data-06h.csv'))
dataset_12h = pd.read_csv(os.path.join(root, 'data-12h.csv'))
dataset_24h = pd.read_csv(os.path.join(root, 'data-24h.csv'))

In [142]:
# display(dataset_06h)
# display(dataset_12h)
# display(dataset_24h)

In [143]:
# print(dataset_06h['agitation'].equals(dataset_06h['agitation-next']))
# print(np.sum(dataset_06h['agitation']))
# print(np.sum(dataset_06h['agitation-next']))

# print(np.where(dataset_06h['agitation'] == 1))
# print(np.where(dataset_06h['agitation-next'] == 1))

print(np.sum(dataset_06h['agitation-four'] == 0),
      np.sum(dataset_06h['agitation-four'] == 1),
      np.sum(dataset_06h['agitation-four'] == 2),
      np.sum(dataset_06h['agitation-four'] == 3))

print(np.sum(dataset_06h['agitation-next'] == 0),
      np.sum(dataset_06h['agitation-next'] == 1))


1400 117 89 18
1752 135


In [144]:
# Agitation Statistics

temp = dataset_06h.copy()
# temp = dataset_12h

temp['agitation'].fillna(0, inplace=True)


print(f"Total number of participants: {temp['id'].nunique()}")
print(f"Total number of participants with at least one agitation episode: {temp[temp['agitation'] == 1]['id'].nunique()}")
print(f"Total number of participants with no agitation episodes: {temp['id'].nunique() - temp[temp['agitation'] == 1]['id'].nunique()}")
print(f"Total number of agitation episodes: {(temp['agitation'] == 1).sum()}")




ids, total_counts = np.unique(temp['id'], return_counts=True)
agitated_ids, agitation_counts = np.unique(temp[temp['agitation'] == 1]['id'], return_counts=True)
agitation_map = dict(zip(agitated_ids, agitation_counts))
unique_dates = temp.groupby('id')['date'].nunique().reindex(ids, fill_value=0).values
participant_summary_df = pd.DataFrame({
    'participant_id': ids,
    'six_hour_sample_count': total_counts,
    'unique_dates': unique_dates,
    'agitation_episodes': [agitation_map.get(pid, 0) for pid in ids]
})
participant_summary_df = participant_summary_df.sort_values(by='agitation_episodes', ascending=False).reset_index(drop=True)
display(participant_summary_df)


values, counts = np.unique(temp.loc[temp['agitation'] == 1, '6h'], return_counts=True)
# values, counts = np.unique(temp.loc[temp['agitation'] == 1, '12h'], return_counts=True)

agitation_temporal_df = pd.DataFrame({
    '6h_time_block': values,
    'agitation_episode_count': counts
})
display(agitation_temporal_df)

Total number of participants: 56
Total number of participants with at least one agitation episode: 27
Total number of participants with no agitation episodes: 29
Total number of agitation episodes: 135


,participant_id,six_hour_sample_count,unique_dates,agitation_episodes
0,73f7c,190,48,33
1,93c14,185,47,12
2,c5031,169,43,12
3,0d5ef,191,49,11
4,16f4b,353,89,11
5,7db78,270,68,8
6,a2849,234,59,6
7,d93d8,210,53,6
8,ec812,288,73,4
9,e4959,262,66,4


,6h_time_block,agitation_episode_count
0,06-12,11
1,12-18,79
2,18-24,45


In [145]:
dataset = dataset_06h
# dataset = dataset_12h


display(dataset['agitation'].unique())
dataset['agitation'].fillna(0, inplace=True)
display(dataset['agitation'].unique())
display(dataset['agitation'].sum())
dataset.drop(['agitation-next', 'agitation-four'], axis=1, inplace=True)


array([ 0., nan,  1.])

array([0., 1.])

np.float64(135.0)

In [146]:
# columns = dataset.columns
# dataset = dataset[columns]
# display(dataset)
# print(dataset.columns.values)

In [147]:
columns_count = ['back-door', 'bathroom', 'bedroom', 'fridge-door',
                 'front-door', 'hallway', 'kitchen', 'lounge']

columns_statistical = ['back-door-count-max', 'back-door-count-mean', 'back-door-count-std', 'back-door-count-sum',
                       'bathroom-count-max', 'bathroom-count-mean', 'bathroom-count-std', 'bathroom-count-sum',
                       'bedroom-count-max', 'bedroom-count-mean', 'bedroom-count-std', 'bedroom-count-sum',
                       'fridge-door-count-max', 'fridge-door-count-mean', 'fridge-door-count-std', 'fridge-door-count-sum',
                       'front-door-count-max', 'front-door-count-mean', 'front-door-count-std', 'front-door-count-sum',
                       'hallway-count-max', 'hallway-count-mean', 'hallway-count-std', 'hallway-count-sum',
                       'kitchen-count-max', 'kitchen-count-mean', 'kitchen-count-std','kitchen-count-sum',
                       'lounge-count-max', 'lounge-count-mean', 'lounge-count-std','lounge-count-sum']

columns_contextual = ['total-events', 'unique-locations', 'active-location-ratio', 'private-to-public-ratio',
                      'location-entropy', 'location-dominance-ratio', 'back-and-forth-count', 'num-transitions']

columns_physiology = ['body-temperature', 'body-weight', 'diastolic-blood-pressure', 'heart-rate',
                      'muscle-mass', 'systolic-blood-pressure', 'total-body-water', 'skin-temperature']

columns_demographics = ['age', 'sex']

columns_labels = ['blood-pressure', 'body-water', 'pulse', 'weight', 'body-temperature-label']

# feature_names = columns_count + columns_statistical + columns_contextual + columns_physiology + columns_demographics + columns_labels
# feature_names = columns_count + columns_statistical + columns_contextual + columns_physiology
feature_names = columns_statistical + columns_contextual + columns_physiology
feature_names = columns_count

print(len(columns_count),
      len(columns_statistical),
      len(columns_contextual),
      len(columns_physiology),
      len(columns_demographics),
      len(columns_labels),
      len(feature_names))



8 32 8 8 2 5 8


In [148]:
# Imputation

import utils_data
importlib.reload(utils_data)
from utils_data import hierarchical_imputation, hierarchical_imputation_columns_to_exclude

print(dataset.shape)

dataset.drop(columns_count + columns_labels + columns_demographics, axis=1, inplace=True)

print(dataset.shape)




print(dataset.isna().sum().sum())

dataset = hierarchical_imputation_columns_to_exclude(dataset, columns_to_exclude=['id',
                                                                                  'date',
                                                                                  '6h',
                                                                                  'agitation'])

print(dataset.isna().sum().sum())

temp = dataset.drop([                                                             'id',
                                                                                  'date',
                                                                                  '6h',
                                                                                  'agitation'], axis=1, inplace=False)
print(temp.isna().sum().sum())


(10790, 67)
(10790, 52)
89869
0
0


In [149]:
# Dataset - Sequential

import utils_data
importlib.reload(utils_data)
from utils_data import segment_dataframe

dataset.fillna(0, inplace=True)

sements, labels = segment_dataframe(dataset, segment_length)

x = np.array(sements)
y = np.array(labels)

# display(x, y)
print(np.sum(y))
print(x.shape)
print(y.shape)

135
(10622, 3, 48)
(10622,)


In [150]:
# ROCKET

name = 'ROCKET'.lower()

Y_TRUES = np.empty([0])
Y_PROBS = np.empty([0])
Y_PREDS = np.empty([0])


if kfold:
    cv = KFold(n_splits=5, shuffle=True, random_state=seed)
    split_iterator = cv.split(x)
else:
    cv = LeavePGroupsOut(n_groups=1)
    split_iterator = cv.split(x, y, groups=p)

for i, (train_idx, test_idx) in enumerate(split_iterator, start=1):
    participant = i if kfold else np.unique(p[test_idx])[0]

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])


    num_samples, time_steps, num_features = x_train.shape
    x_train_flat = x_train.reshape(-1, num_features)
    x_test_flat = x_test.reshape(-1, num_features)

    scaler = StandardScaler()
    scaler.fit(x_train_flat)
    x_train_scaled = scaler.transform(x_train_flat)
    x_test_scaled = scaler.transform(x_test_flat)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train_scaled)
    x_train_scaled = normalizer.transform(x_train_scaled)
    x_test_scaled = normalizer.transform(x_test_scaled)

    x_train = x_train_scaled.reshape(num_samples, time_steps, num_features)
    x_test = x_test_scaled.reshape(x_test.shape[0], time_steps, num_features)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))


    rocket = Rocket(num_kernels=1_000)
    rocket.fit(x_train)
    x_train_transformed = rocket.transform(x_train)
    x_test_transformed = rocket.transform(x_test)


    # ----- Gradient Boosting Classifier
    model = GradientBoostingClassifier(
        n_estimators=100,  # match LightGBM
        learning_rate=0.001,  # match LightGBM
        max_depth=4,  # similar to LightGBM default tree depth
        subsample=1.0,  # default
        random_state=seed
    )
    if weighted:
        model.fit(x_train_transformed, y_train, sample_weight=compute_sample_weight(class_weight='balanced', y=y_train))
    else:
        model.fit(x_train_transformed, y_train)

    y_probs = model.predict_proba(x_test_transformed)[:, 1]
    y_preds = model.predict(x_test_transformed)
    

    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS = np.append(Y_PROBS, y_probs)
    Y_PREDS = np.append(Y_PREDS, y_preds)


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics
auc_roc = roc_auc_score(Y_TRUES, Y_PROBS)
precision, recall, _ = precision_recall_curve(Y_TRUES, Y_PROBS)
auc_pr = auc(recall, precision)
acc = accuracy_score(Y_TRUES, Y_PREDS)
pre = precision_score(Y_TRUES, Y_PREDS)
rec = recall_score(Y_TRUES, Y_PREDS)
f1 = f1_score(Y_TRUES, Y_PREDS)

# Confusion matrix: [ [TN, FP], [FN, TP] ]
tn, fp, fn, tp = confusion_matrix(Y_TRUES, Y_PREDS).ravel()

# Sensitivity = Recall = TP / (TP + FN)
sensitivity = tp / (tp + fn)

# Specificity = TN / (TN + FP)
specificity = tn / (tn + fp)

results = {
    'auc-roc': f"{auc_roc:.4f}",
    'auc-pr': f"{auc_pr:.4f}",
    'accuracy': f"{acc:.4f}",
    'precision': f"{pre:.4f}",
    'recall': f"{rec:.4f}",
    'f1-score': f"{f1:.4f}",
    'sensitivity': f"{sensitivity:.4f}",
    'specificity': f"{specificity:.4f}",
    'tn': str(int(tn)),
    'fp': str(int(fp)),
    'fn': str(int(fn)),
    'tp': str(int(tp))
}


results = pd.DataFrame([results])
results.to_csv(os.path.join(results_dir, name + '.csv'), index=False)
display(results)

1 8497 2125
2 8497 2125
3 8498 2124
4 8498 2124
5 8498 2124


,auc-roc,auc-pr,accuracy,precision,recall,f1-score,sensitivity,specificity,tn,fp,fn,tp
0,0.8520,0.1103,0.9038,0.0920,0.7407,0.1637,0.7407,0.9059,9500,987,35,100
